In [34]:
import requests
import json

In [35]:
AC_URL = "http://localhost:6380/query"

In [36]:
def generate_trapi_query(input_node_type,output_node_type,input_curies,predicate,input_is_subject=True):
    envelope = {"message":{"query_graph": {"nodes":{"input":{},"output":{}}, "edges":{"edge_0":{}}}}}
    input_node = envelope["message"]["query_graph"]["nodes"]["input"]
    input_node["categories"] = [input_node_type]
    input_node["ids"] = ["uuid:1"]
    input_node["member_ids"] = input_curies
    input_node["set_interpretation"]="MANY"
    output_node = envelope["message"]["query_graph"]["nodes"]["output"]
    output_node["categories"] = [output_node_type]
    query_edge = envelope["message"]["query_graph"]["edges"]["edge_0"]
    if input_is_subject:
        query_edge["subject"] = "input"
        query_edge["object"] = "output"
    else:
        query_edge["subject"] = "output"
        query_edge["object"] = "input"
    query_edge["predicates"]  = [ predicate ]
    #query_edge["knowledge_type"] = "inferred"
    return envelope

In [37]:
#Phenotype to Disease
input_type = "biolink:PhenotypicFeature"
output_type = "biolink:Disease"
predicate = "biolink:has_phenotype"
input_is_subject = "False"

In [38]:
#Leigh Disease?
leigh_phenotypes = ["HP:0000739", #| Anxiety, 
                    "HP:0001288", # | Gait disturbance, 
                    "HP:0001252", # | Hypotonia, 
                    "HP:0001250", # | Seizure, 
                    "HP:0000750", # | Delayed speech and language development, 
                    "HP:0002378", # | Hand tremor, 
                    "HP:0002019", # | Constipation, 
                    "HP:0007146"] # | Bilateral basal ganglia lesions

In [40]:
leigh_query = generate_trapi_query( input_type, output_type, leigh_phenotypes, predicate, input_is_subject)
response = requests.post( AC_URL, json = leigh_query )
print( response.status_code )

200


In [42]:
resp = response.json()
for result in resp["message"]["results"]:
    nb = result["node_bindings"]["output"][0]["id"]
    name = resp["message"]["knowledge_graph"]["nodes"][nb]["name"]
    print(nb, name)
    

MONDO:0009142 dystonia with Ringbinden
MONDO:0013962 hereditary spastic paraplegia 53
MONDO:0011468 hereditary motor and sensory neuropathy, Okinawa type
MONDO:0010319 syndromic X-linked intellectual disability Hedera type
MONDO:0014553 Tenorio syndrome
MONDO:0012789 dystonia 16
MONDO:0013674 neurodegeneration with brain iron accumulation 4
MONDO:0010496 X-linked intellectual disability-short stature-overweight syndrome
MONDO:0010305 creatine transporter deficiency
MONDO:0013578 DYRK1A-related intellectual disability syndrome
MONDO:0060510 Cohen-Gibson syndrome
MONDO:0008006 Mobius syndrome
MONDO:0030047 microcephaly, developmental delay, and brittle hair syndrome
MONDO:0014686 short stature, microcephaly, and endocrine dysfunction
MONDO:0100218 arthrogryposis multiplex congenita 5
MONDO:0010500 intellectual disability, X-linked, syndromic 33
MONDO:0010406 chromosome Xp11.22 duplication syndrome
MONDO:0010413 intellectual disability, X-linked 95
MONDO:0020853 encephalitis/encephalopath

In [43]:
# Phenotype to Gene
input_type = "biolink:PhenotypicFeature"
output_type = "biolink:Gene"
predicate = "biolink:genetically_associated_with"
input_is_subject = "True"

In [44]:
leigh_query = generate_trapi_query( input_type, output_type, leigh_phenotypes, predicate, input_is_subject)
response = requests.post( AC_URL, json = leigh_query )
print( response.status_code )

200


In [45]:
resp = response.json()
for result in resp["message"]["results"]:
    nb = result["node_bindings"]["output"][0]["id"]
    name = resp["message"]["knowledge_graph"]["nodes"][nb]["name"]
    print(nb, name)

NCBIGene:93986 FOXP2
NCBIGene:53335 BCL11A
NCBIGene:85358 SHANK3
NCBIGene:9378 NRXN1
NCBIGene:2077 ERF
NCBIGene:1806 DPYD
NCBIGene:26040 SETBP1
NCBIGene:26047 CNTNAP2
NCBIGene:50484 RRM2B
NCBIGene:284058 KANSL1
NCBIGene:113189 CHST14
NCBIGene:2903 GRIN2A
NCBIGene:5728 PTEN
NCBIGene:3737 KCNA2
NCBIGene:3954 LETM1
NCBIGene:4204 MECP2
NCBIGene:18 ABAT
